In [2]:
import polars as pl
import re
import emoji


In [3]:
df=pl.read_csv("./dataset/data_cleaned.csv")
df.head()

question,answer
str,str
""" هل نسبة المياه ببنين أقل منها…","""نعم"""
"""هل متوسط العمر بالهند أقل منه …","""نعم"""
"""هل فرس البحر و النعامة ينتميان…","""لا"""
"""هل الكثافة السكانية في غانا أع…","""لا"""
"""هل معدل البطالة بالصين أقل منه…","""نعم"""


In [4]:
def preprocess_arabic_text(text: str, is_tydiqa_goldp: bool = False) -> str:
    if text is None: return "" 
    
    text = emoji.replace_emoji(text, replace="")
    
    if not is_tydiqa_goldp:
        text = re.sub(r'<.*?>', '', text)
        
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '[EMAIL]', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '[URL]', text)
    text = re.sub(r'@[a-zA-Z0-9_]+', '[MENTION]', text)
    
    text = re.sub(r'[\u064B-\u0652\u0640]', '', text)
    
    pattern_non_allowed = r'([^a-zA-Z0-9\u0621-\u064a\u0660-\u0669\(\)\[\]\{\}\s])'
    text = re.sub(pattern_non_allowed, r' \1 ', text)

    text = re.sub(r'([a-zA-Z\u0621-\u064a]+)(\d+)', r'\1 \2', text)
    text = re.sub(r'([a-zA-Z\u0621-\u064a]+)([\u0660-\u0669]+)', r'\1 \2', text)
    text = re.sub(r'(\d+)([a-zA-Z\u0621-\u064a]+)', r'\1 \2', text)
    text = re.sub(r'([\u0660-\u0669]+)([a-zA-Z\u0621-\u064a]+)', r'\1 \2', text)

    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

cols_to_process = ["question", "answer"]
df = df.with_columns([
    pl.col(col).map_elements(preprocess_arabic_text, return_dtype=pl.Utf8) 
    for col in cols_to_process
])



In [5]:
df["answer"].tail()

answer
str
"""{ ' text ' : array([ ' الأمم ا…"
"""{ ' text ' : array([ ' شمال شر…"
"""{ ' text ' : array([ ' تشاد وج…"
"""{ ' text ' : array([ ' إثيوبيا…"
"""{ ' text ' : array([ ' الأول م…"
"""{ ' text ' : array([ ' منذ قبي…"
"""{ ' text ' : array([ ' 2005 ' …"
"""{ ' text ' : array([ ' عام 198…"
"""{ ' text ' : array([ ' العميد …"


In [6]:
garbage_pattern = r"array\(|dtype=|\{'text':|\[|\]"

df_clean = df.filter(
    ~pl.col("answer").str.contains(garbage_pattern) &
    pl.col("answer").is_not_null() &
    (pl.col("answer").str.strip_chars() != "")
)

print(f"Original size: {len(df)}")
print(f"Clean size: {len(df_clean)}")
print(f"Removed {len(df) - len(df_clean)} rows.")

print(df_clean.tail(1)[0, 1]) 

Original size: 1061374
Clean size: 1052308
Removed 9066 rows.
في هذا المكان عادة تكون كتلة دهنية . ولكن لا يمكن التأكد بدون فحص طبي 0 2016 - 03 - 10 07 : 15 : 23 الدكتور باسم مرقص / اسئلة - طبية / الاورام - الخبيثة - والحميدة / الاحظ - وجود - ورم - بالفخذ - الايسر - بدون - الاحساس - بالم - ولكنه - 710147 الدكتور باسم مرقص جراحة عامة


In [7]:

garbage_pattern = r"array\(|dtype=|\{'text':|\[|\]"


df_clean = df.filter(
    ~pl.col("answer").str.contains(garbage_pattern)
)

print(f"Original size: {len(df)}")
print(f"Clean size: {len(df_clean)}")
print(f"Removed {len(df) - len(df_clean)} rows.")

Original size: 1061374
Clean size: 1058708
Removed 2666 rows.


In [8]:
df_clean.tail()[0,0]

'التهاب حاد في البنكرياس تم عمل ct & MRI تبين تضيق في الجزء الشمالي من الكبد وتوسع بالقنوات المراري مع وجودr stones وهناك كتلة غير متجانسة عالكللية هل ينصح بعمل استئصال نص كبد'

In [9]:
df_clean.write_csv("./dataset/data_preprocessed.csv")